# Final project (part 1)

Team leader: Zhaoliang Zheng

Team member: Anqi Shen, Enbo Yu

#### step1: data preprocessing
#### step2: data processing
#### step3: feature extraction and vectorization
#### step4: train, test validation set split
#### step5: building classifiers
#### step6: hypertuing

In [83]:
import os
ROOTDIR = os.path.abspath(os.path.dirname('__file__'))
print(ROOTDIR)

C:\Users\zzl\data\final project


In [84]:
import pandas as pd
#Load dataset and check it
df = pd.read_csv(os.path.join(ROOTDIR, 'fake_or_real_news.csv'))
df.head()

,Unnamed: 0,title,text,label,title_vectors
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,[ 1.1533764e-02 4.2144405e-03 1.9692603e-02 ...
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,[ 0.11267698 0.02518966 -0.00212591 0.021095...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,[ 0.04253004 0.04300297 0.01848392 0.048672...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,[ 0.10801624 0.11583211 0.02874823 0.061732...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,[ 1.69016439e-02 7.13498285e-03 -7.81233795e-...


## Step1: DATA PREPROCESSING

In [85]:
#define a binarize function and binrize the label
def binarize(word):
    if word== 'REAL':
        return 0
    else:
        return 1
df.label= df.label.apply(binarize)

In [86]:
#in this step, I'm gonna drop the first column which is meaningless here
df.drop('Unnamed: 0', axis=1, inplace=True)

In [87]:
#see the result
df.head()

,title,text,label,title_vectors
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1,[ 1.1533764e-02 4.2144405e-03 1.9692603e-02 ...
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1,[ 0.11267698 0.02518966 -0.00212591 0.021095...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0,[ 0.04253004 0.04300297 0.01848392 0.048672...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1,[ 0.10801624 0.11583211 0.02874823 0.061732...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0,[ 1.69016439e-02 7.13498285e-03 -7.81233795e-...


## Step2: DATA PROCESSING:TOKENIZE & CLEAN THE TEXT

In [88]:
import nltk
from nltk.corpus import gutenberg
from gensim import models
from string import punctuation
from nltk.corpus import stopwords

In [89]:
#load raw title and text dataset into different varibles
texts = df.text.values
titles = df.title.values

In [90]:
#simply tokenize the text and title
tokenized_text = [nltk.word_tokenize(text) for text in texts]
tokenized_title = [nltk.word_tokenize(title) for title in titles]

check the high frequency words in top 20

we find out that there are lots of stop words, punctuations and useless mark in the tokenized text

In [91]:
from collections import Counter

token_counter = Counter(token.lower() for sentence in tokenized_title for token in sentence)
top20 = token_counter.most_common()[:20]
for i, t in enumerate(top20):
    print('{:>2}.{:>11}  freq: {:>7}'.format(i+1, t[0], t[1]))

 1.        the  freq:    2111
 2.         to  freq:    1714
 3.          :  freq:    1618
 4.          ,  freq:    1409
 5.          ’  freq:    1380
 6.         in  freq:    1161
 7.         of  freq:    1091
 8.      trump  freq:    1079
 9.        for  freq:     799
10.         on  freq:     764
11.          s  freq:     741
12.          a  freq:     736
13.    clinton  freq:     709
14.         is  freq:     700
15.        and  freq:     693
16.    hillary  freq:     580
17.         's  freq:     566
18.          ?  freq:     554
19.      obama  freq:     375
20.          '  freq:     373


we define a clean_advance function to clean the text and title

In [92]:
sw = stopwords.words('english')
def clean_advance(tokenized_list, sw):
    new_list = []
    for doc in tokenized_list:
        new_list.append([token.lower() for token in doc if token.lower() not in sw
                        and token.lower() != "“" and token.lower() != "”"
                        and token.lower() != "‘" and token.lower() != "’"
                        and token.lower() != "``" and token.lower() != "''"
                        and token.lower() != "—"  and token.lower() != "–"
                        and token.lower() !="," and token.lower() != "."
                        and token.lower() != ":" and token.lower() != "?"
                        and token.lower() != "(" and token.lower() != ")"
                        and token.lower() != "'s" and token.lower() != "n't"
                        and token.lower() != "'d" and token.lower() != "'ve"
                        and token.lower() != "!" and token.lower() != "'"
                        and token.lower() != "&" and token.lower() != "*"
                        and token.lower() != "..." and token.lower() != "…" 
                        and token.lower() != "#" and token.lower() != "-"
                        and token.lower() != "[" and token.lower() != "]" 
                        and token.lower() != "%" and token.lower() !="|" 
                        and token.lower() != ";" ])
    return new_list

In [102]:
# Remove punctuations and stopwords,lower-case and other useless mark from tokenized title and text
cleaned_title = clean_advance(tokenized_title, sw)
cleaned_text = clean_advance(tokenized_text, sw)

In [94]:
token_counter = Counter(token.lower() for sentence in cleaned_text for token in sentence)
top20 = token_counter.most_common()[:20]
for i, t in enumerate(top20):
    print('{:>2}.{:>11}  freq: {:>7}'.format(i+1, t[0], t[1]))

 1.      trump  freq:   21906
 2.       said  freq:   21162
 3.    clinton  freq:   17237
 4.      would  freq:   12928
 5.     people  freq:   11598
 6.        one  freq:   11295
 7.        new  freq:    9263
 8.      state  freq:    8744
 9.  president  freq:    8509
10.       also  freq:    8206
11.      obama  freq:    8121
12.   campaign  freq:    7642
13.         us  freq:    7511
14.    hillary  freq:    7127
15.       like  freq:    7038
16.      could  freq:    6705
17.       time  freq:    6432
18.       even  freq:    6424
19.     states  freq:    6142
20.       many  freq:    5847


### 1：Extend title with text

In [113]:
import copy
c_extend=copy.deepcopy(cleaned_title)

In [114]:
for i in range(texts.shape[0]):
   c_extend[i].extend(cleaned_text[i])

In [132]:
import pandas as pd
clean_extend=pd.DataFrame(data=c_extend)
clean_extend.to_csv('clean_extend.csv',index=False,header=False)

in this way we only combine title and text and put into a list

essentially we just parallelly load more information into our text

then we do verterization and feature engineering to the clean_extend dataset

### 2：Extend title with text

In [131]:
import pandas as pd
cleantitle=pd.DataFrame(data=cleaned_title)
cleantext=pd.DataFrame(data=cleaned_text)
cleantitle.to_csv('clean_title.csv',index=False,header=False)
cleantext.to_csv('clean_text.csv',index=False,header=False)

or we can just seperatly output our cleaned title and text 

then seperatly do verterization and feature engineering to these two datasets 

In [133]:
cleaned_title[0]

['smell', 'hillary', 'fear']